# Nou HomolWat DB

In [8]:
from tqdm import notebook


La llista de PDBs a partir dels PFAMS

In [9]:
# https://github.com/williamgilpin/pypdb
from pypdb import *
pfamdict = {
    "classA": "PF00001",
    "classB": "PF00002",
    "classC": "PF00003",
    "frizzled": "PF01534",}

pdbids = []
for fam, pfam in pfamdict.items():
    pdbids.extend(Query(pfam, query_type="pfam").search())

#pdbids

## Agafar info del PDB: com truncar les prote�nes de fusi�

Per cada PDBID cal buscar les polymer entities (1 o m�s, que equivalen a les chains). Obrim un PDB com a exemple:

Obtenim una llista de n�meros que ens indiquen les cadenes proteiques i els fitxers seg�ents a obrir (un per cadena). Obrim el primer:

Ara extraiem els rangs que corresponen a la proteina. Cal verificar que �s el del GPCR i no el de la porte�na de fusi�

In [12]:
import pandas as pd
import re
import requests
import json

all_info, l_seg_pdb, l_error, l_seg = [], [], [], []
clas = ""
for pdb in notebook.tqdm(pdbids):
    url_pdb=f"https://data.rcsb.org/rest/v1/core/entry/{pdb.lower()}"
    response = requests.get(url_pdb)
    data = json.loads(response.text)
    try:
        res = data["pdbx_vrpt_summary"]["pdbresolution"]
    except:
        res = ""
    meth = data["rcsb_entry_info"]["experimental_method"]
    ids = data["rcsb_entry_container_identifiers"]["polymer_entity_ids"]
    for id in ids:
        url = f'https://data.rcsb.org/rest/v1/core/polymer_entity/{pdb.lower()}/{id}'
        response = requests.get(url)
        data = json.loads(response.text)
        #chain = data["entity_poly"]["pdbx_strand_id"]# la chain que correspon al polymer entity  
        chain_pdb = data["rcsb_polymer_entity_container_identifiers"]["asym_ids"]
        chain_auth = data["rcsb_polymer_entity_container_identifiers"]["auth_asym_ids"]
        print(pdb, chain_pdb, chain_auth)
        for i, ch in enumerate(chain_pdb):
            mapping = f'https://data.rcsb.org/rest/v1/core/polymer_entity_instance/{pdb.lower()}/{ch}' 
            response = requests.get(mapping)
            data_map = json.loads(response.text)
            auth_map = data_map["rcsb_polymer_entity_instance_container_identifiers"]["auth_to_entity_poly_seq_mapping"]# ["852","853","854","855","856","
            try:
                l_rcsb = data["rcsb_polymer_entity_align"]
                for rcsb in l_rcsb: 
                    # uniprot
                    acc=rcsb['reference_database_accession']
                    url_uni = f'https://www.uniprot.org/uniprot/{acc}'
                    response = requests.get(url_uni)
                    data_uni = response.text
                    search = re.search('<span property="alternateName">(.+?)</span>', data_uni)            
                    result = re.search('>(.*)<', str(search.group()))
                    entry = str(result.group(1)) 
                    entry = entry[1:-1]
                    # extreiem el pfam 
                    search = re.search("https://pfam.xfam.org/family/.*", data_uni)    
                    search = str(search.group())
                    l_search = search.split(">")
                    l_search = l_search[0].split("/")
                    pfam = l_search[-1][:-1]
                    if pfam in pfamdict.values():
                        clas = "receptor"
                    else:
                        clas = "fusion protein"
                    #Segment region
                    l_rscb_fea = data["rcsb_polymer_entity_feature"] 
                    for rscb_fea in l_rscb_fea:
                            # el receptor comen�a un segment a ref_beg_seq_id 
                            l_reg = rcsb['aligned_regions']
                            for reg in l_reg:
                                res_ini_pdb = reg['entity_beg_seq_id'] - 1  # 1 --: 0 
                                res_end_pdb = res_ini_pdb + reg['length'] - 1 
                                res_ini_auth = auth_map[res_ini_pdb] 
                                res_end_auth = auth_map[res_end_pdb] 
                                l_seg.append(f"{res_ini_pdb}-{res_end_pdb}")
                                l_seg_pdb.append(f"{res_ini_auth}-{res_end_auth}")
                            all_info.append([pdb,res,meth,id,acc,entry,pfam,chain_auth[i][0],",".join(l_seg_pdb), clas])
                            l_info, l_seg_pdb, l_seg = [], [], []
                            clas = ""
            except Exception as e:
                all_info.append([pdb,res,meth,id,"","","",chain_auth[i][0],"","", ""])
                l_info, l_seg_pdb, l_seg = [], [], []
                clas = ""
                if pdb not in l_error:
                    l_error.append(pdb) 

  0%|          | 0/657 [00:00<?, ?it/s]

1EDS ['A'] ['A']
1EDV ['A'] ['A']
1EDW ['A'] ['A']
1F88 ['A', 'B'] ['A', 'B']
1FDF ['A'] ['A']
1GZM ['A', 'B'] ['A', 'B']
1HLL ['A'] ['A']
1HO9 ['A'] ['A']
1HOD ['A'] ['A']
1HOF ['A'] ['A']
1HZN ['A'] ['A']
1HZX ['A', 'B'] ['A', 'B']
1JFP ['A'] ['A']
1L4T ['A'] ['A']
1L9H ['A', 'B'] ['A', 'B']
1LN6 ['A'] ['A']
1LVQ ['A'] ['A']
1LVR ['A'] ['A']
1RY1 ['A'] ['E']
1RY1 ['B'] ['A']
1RY1 ['C'] ['M']
1RY1 ['D'] ['N']
1RY1 ['E'] ['O']
1RY1 ['F'] ['P']
1RY1 ['G'] ['Q']
1RY1 ['H'] ['R']
1RY1 ['I'] ['C']
1RY1 ['J'] ['D']
1RY1 ['K'] ['B']
1RY1 ['L'] ['U']
1RY1 ['M'] ['W']
1RY1 ['N'] ['S']
1U19 ['A', 'B'] ['A', 'B']
2CSA ['A'] ['A']
2DCO ['A'] ['A']
2G87 ['A', 'B'] ['A', 'B']
2HPY ['A', 'B'] ['A', 'B']
2I35 ['A'] ['A']
2I36 ['A', 'B', 'C'] ['A', 'B', 'C']
2I37 ['A', 'B', 'C'] ['A', 'B', 'C']
2J28 ['A'] ['0']
2J28 ['B'] ['1']
2J28 ['C'] ['2']
2J28 ['D'] ['3']
2J28 ['E'] ['4']
2J28 ['F'] ['7']
2J28 ['G'] ['8']
2J28 ['H'] ['9']
2J28 ['I'] ['A']
2J28 ['J'] ['B']
2J28 ['K'] ['C']
2J28 ['L'] ['D']
2J28 [

In [ ]:
l_error = list(dict.fromkeys(l_error))
print(l_error)

In [13]:
import numpy as np

print(all_info[0])
df = pd.DataFrame(all_info, columns=["PDB accesion", "Resolution", "Method", "Id", "Uniprot accesion", "Uniprot entry", "Pfam accesion", "Chain", "Segments Pdb", "Classification", "What"])
conditions = [
    (df["Pfam accesion"] == "PF00001"),
    (df["Pfam accesion"] == "PF00002"),
    (df["Pfam accesion"] == "PF00003"),
    (df["Pfam accesion"] == "PF01534")
] 
df = df.drop(["What"],axis = 1)
values = ["classA", "classB", "classC", "frizzled"]
df["Family"] = np.select(conditions, values)
all_info_rec = df[(df["Classification"] == "receptor") & (df["Method"] != "NMR") & (df["Family"] != "0") ] 
all_info_rec.to_csv("HomolWat_db.csv", index=False)
df.to_csv("GPCRstdb.csv", index=False)
df

['1EDS', '', 'NMR', '1', 'P02699', 'OPSD_BOVIN', 'PF00001', 'A', '1-31', 'receptor']


,PDB accesion,Resolution,Method,Id,Uniprot accesion,Uniprot entry,Pfam accesion,Chain,Segments Pdb,Classification,Family
0,1EDS,,NMR,1,P02699,OPSD_BOVIN,PF00001,A,1-31,receptor,classA
1,1EDS,,NMR,1,P02699,OPSD_BOVIN,PF00001,A,1-31,,classA
2,1EDS,,NMR,1,P02699,OPSD_BOVIN,PF00001,A,1-31,,classA
3,1EDS,,NMR,1,P02699,OPSD_BOVIN,PF00001,A,1-31,,classA
4,1EDV,,NMR,1,P02699,OPSD_BOVIN,PF00001,A,1-34,receptor,classA
...,...,...,...,...,...,...,...,...,...,...,...
18240,7EVW,3.22,EM,5,P62873,GBB1_HUMAN,PF00400,B,1-340,,0
18241,7EVW,3.22,EM,5,P62873,GBB1_HUMAN,PF00400,B,1-340,,0
18242,7EVW,3.22,EM,5,P62873,GBB1_HUMAN,PF00400,B,1-340,,0
18243,7EVW,3.22,EM,5,P62873,GBB1_HUMAN,PF00400,B,1-340,,0


El pas seg�ent �s anar a PyMOL / mdanalysis. Fer fetch del PBD (o PDB REDO). Eliminar amino acids fora dels segments determinats

In [7]:
from pkg_resources import Requirement, resource_filename
import pymol
import os
import glob
import sys
#import a_pdb2ball
from clize import run
import wget
import urllib
import pandas as pd

allres = "ALA+ARG+ASP+GLU+THR+TYR+HIS+LEU+ILE+VAL+PRO+GLY+TRP+LYS+PHE+ASN+GLN+MET+CYS+SER+ACE+NH2+DI7+DI8"
wat_ions = "HOH+NA+CL+ZN+SO4"
dirs = ["FULL", "CHAIN", "REC_MOL_WAT"]

#PREPARE THE CHAINS, UNIPROT ACC, EXP 
def get_pdb_chains():
    """Gets the list of PDBs and their chains from PDBLIB"""
    all_info_rec = pd.read_csv("E:\Work\HomolWat\HomolWat_db.csv", sep=";")
    pdb_list = all_info_rec["PDB accesion"].tolist()
    d_chains = {}
    for pdb in pdb_list: 
        f_chains = all_info_rec[all_info_rec["PDB accesion"] == pdb]
        d_chains[pdb] = f_chains["Chain"].to_list()
    #d_chains = dict(list(zip(all_info_rec["PDB accesion"], all_info_rec["Chain"])))
    d_uniprot = dict(list(zip(all_info_rec["PDB accesion"], all_info_rec["Uniprot entry"])))
    d_expmet = dict(list(zip(all_info_rec["PDB accesion"], all_info_rec["Method"])))
    d_segment = dict(list(zip(all_info_rec["PDB accesion"], all_info_rec["Segments Pdb"])))
    return pdb_list, d_chains, d_uniprot, d_expmet, d_segment

def prepare_FULL(pdb_list):
    """Prepares the direcory full.
    PDB_LIST is the list of PDBs in PDBLIB;
    pdb_pepts contains the chain mappings;
    uniprot contains the ids from PDBLIB"""
    # import shutil
    import os

    new_pdbs = []  # xxxx_A
    os.chdir(f"{PDB_PATH}/PDB/FULL")
    for pdb in pdb_list:  # these are xxxx(.pdb)
        print(pdb, "in pdb_list")
        # DO SOMETHING ONLY IF PDB IS NEW!!!
        if (
            (not os.path.exists(pdb.lower() + "_final.pdb"))
            and (not os.path.exists(pdb.lower() + "_updated.cif"))
            and (not os.path.exists(pdb.lower() + ".cif"))
        ):
            found_pdb = False
            if d_expmet[pdb] == "X-ray":
                try:
                    url = (
                        f"https://pdb-redo.eu/db/{pdb.lower()}/{pdb.lower()}_final.pdb"
                    )
                    wget.download(url)
                    found_pdb = True
                    print(f"\nDownloading PDB-REDO structure {pdb}\n")
                except urllib.error.HTTPError:
                    pass
            if not found_pdb:
                try:
                    url = f"http://www.ebi.ac.uk/pdbe/entry-files/download/{pdb.lower()}_updated.cif"
                    wget.download(url)
                    found_pdb = True
                    print(f"\nDownloading updated CIF PDBe structure {pdb}\n")
                except urllib.error.HTTPError:
                    try:
                        url = f"http://www.ebi.ac.uk/pdbe/entry-files/download/{pdb.lower()}.cif"
                        wget.download(url)
                        found_pdb = True
                        print(f"\nDownloading updated CIF PDBe structure {pdb}\n")
                        # http://www.ebi.ac.uk/pdbe/entry-files/download/pdb2rh1.ent
                        # prevents strange errors when a PDB cannot be retrieved (it will the next run)
                        #
                    except urllib.error.HTTPError:
                        print("Not found!")
                        sys.exit()
            if found_pdb:
                new_pdbs.append(pdb)
        else:
            print("Already processed. Remove file from ./FULL to regenerate it")

    # ends in FULL
    os.chdir("..")
    print(20 * "-" + "\n")
    print("        --- FINISHED --- directory FULL ---")
    return new_pdbs

def prepare_CHAIN(new_pdbs, d_chains, d_uniprot):
    import glob

    print(new_pdbs)
    os.chdir(f"{PDB_PATH}/PDB/FULL")
    pdbs_in_cif = [pdb[:4] for pdb in glob.glob("*.cif")]
    print("pdbs_in_cif", pdbs_in_cif)
    pdbs_from_pdbredo = [pdb[:4] for pdb in glob.glob("*_final.pdb")]
    print("pdbs_from_pdbredo", pdbs_from_pdbredo)

    os.chdir(f"{PDB_PATH}/PDB")
    for pdb in new_pdbs:  # these are 1f88_updated.cif
        print(pdb)
        pdb = pdb[0:4].lower()
        # print ' Chains', d_chains[pdb]
        pymol.cmd.do("cd " + "./FULL")
        print("<><>" + 20 * "-" + "\n" + pdb)
        if pdb.lower() in pdbs_in_cif:
            pymol.cmd.do("load " + pdb.lower() + "_updated.cif")
            pymol.cmd.do(f"save {pdb}.pdb, {pdb}")
            pymol.cmd.do("delete " + pdb)
            # os.remove(pdb + ".cif")
            pymol.cmd.do("load " + pdb + ".pdb")
        elif pdb.lower() in pdbs_from_pdbredo:
            pymol.cmd.do("load " + pdb.lower() + "_final.pdb")
            pymol.cmd.do(f"save {pdb}.pdb, {pdb}_final")
            pymol.cmd.do("delete " + pdb)
            # os.remove(pdb + "_final.pdb")
            pymol.cmd.do("load " + pdb + ".pdb")
        else:
            print("No PDB to load")
            sys.exit()
        pymol.cmd.do("cd " + "../CHAIN")
        #l_ch = d_chains[pdb.upper()].split(",")
        l_ch = d_chains[pdb.upper()]
        for ch in l_ch:
            # print(ch)
            # Break the PDB in CHAINs
            # This is for peptides, which have their own chain!!
            # with_pept = False
            # for pdb_pept in pdb_pepts:
            #     if pdb in pdb_pept[0]:
            #         with_pept = True
            #         print(pdb_pept[1])  # ????
            #         if ch in pdb_pept[1]:  # avoid unknown chains as keys
            #             cmd_txt = "select sele_ch, chain {}+{}".format(
            #                 pdb_pept[1][ch], ch
            #             )
            #             pymol.cmd.do(cmd_txt)
            #             break
            print("----->", pdb)
            # THE NEXT FOUR LINES WERE INTRODUCED RECENTLY AND HAVE NOT BEEN TESTED
            # pymol.cmd.do("select prot, resn " + allres + " and " + pdb.lower())
            # pymol.cmd.do("select close, (br. prot around 5) and " + pdb.lower())
            # pymol.cmd.do("select far, not (byres (close or prot) around 5) and not (close or prot) and " + pdb.lower())
            # pymol.cmd.do("remove far")
            # if not with_pept:  # general case
            pymol.cmd.do("select sele_ch, chain " + ch)
            # print pdb, ch, d_uniprot[pdb]  # for testing
            pymol.cmd.do(
                "save " + pdb.upper() + "_" + ch + "_" + d_uniprot[pdb.upper()] + ".pdb, sele_ch"
            )
            if pdb.upper() == "4EIY" and ch == "A":
                pymol.cmd.do(
                "save ref_4EIY_A.pdb, sele_ch"
                )

        pymol.cmd.do("delete *")
        pymol.cmd.do("cd " + "..")

    # ends in FULL
    print(20 * "-" + "\n")
    print("        --- FINISHED --- directory CHAINS ---")

# Check
def prepare_REC(new_pdbs, d_segments):
    """Actually prepares  REC_MOL_WAT....
    Prepare PDBs with/without WAT, MOL all supperposed to "refpdb"
    "new_pdbs" is a list"""
    l_error_wat = []
    print(os.getcwd())
    os.chdir(f"{PDB_PATH}/PDB/CHAIN")
    pdb_c = glob.glob("????_*_*.pdb")
    #pdb_c = ['2RH1_A_ADRB2.pdb', '4DKL_A_OPRM.pdb']  # for testing
    rms_a, rms_s, rms_pdb = [], [], []
    refpdb = "ref_4EIY_A"  # this is the reference structure for superposition
    pymol.cmd.do("load " + refpdb + ".pdb")
    os.chdir(f"{PDB_PATH}/PDB")
    # with open(
    #     resource_filename(Requirement.parse("lmcdb"), "lmcdb-admin/fusion_residues.txt")
    # ) as file_fusion:
    #     lines = file_fusion.readlines()
    # fusion_dict = {}
    # for line in lines[1:]:
    #     pdb, res_string = line.strip().split()
    #     fusion_dict[pdb] = res_string

    # Loop stats at dir MAIN
    #print("new_pdbs", new_pdbs)
    d_waters = {}
    for pdbname in pdb_c:  # these are xxxx_A.pdb
        shrt_pdbn = pdbname[:-4]  # strip extension
        # if 1 ==1:  # for testing; replace if
        #if shrt_pdbn[:4].upper() in new_pdbs:  # for comparison take only pdbid
        print("#############", shrt_pdbn, pdbname)
        pymol.cmd.do("cd " + "./CHAIN")
        pymol.cmd.do("load " + pdbname)
        print(pdbname)
        # save 1 single chain without lysozyme, BRIL ....
        # EXCEPTIONs: BRIL/T4L at the N-terminus ...
        chain = pdbname[5]
        segment = d_segments[pdbname[0:4]]
        segment = segment.replace(",", "+")
        print(segment)
        try:
            #select not resi 1-352 and (segid A and 4DKL_A_OPRM_MOUSE)
            pymol.cmd.do("select solvent and "
                + shrt_pdbn)
            wat = pymol.cmd.count_atoms("sele")
            print(wat)
            d_waters[pdbname[0:4]] = int(wat)
            print(d_waters)
            pymol.cmd.do(
                "select not resi " 
                + segment
                + " and not hetatm and "
                + shrt_pdbn 
            )  # before 900-5000
            pymol.cmd.do("remove sele")
            # remove water + ion that are very far
            pymol.cmd.do("select prot, resn " + allres + " and " + shrt_pdbn)
            pymol.cmd.do("select close, (br. prot around 5) and " + shrt_pdbn)
            pymol.cmd.do(
                "select far, not (byres (close or prot) around 5) and not (close or prot) and "
                + shrt_pdbn
            )
            pymol.cmd.do("remove far")
            # Superposition of structures
            rms_data_super = pymol.cmd.super(
                shrt_pdbn + "////CA" + " and chain " + chain, refpdb + "////CA"
            )
            rms_data_align = pymol.cmd.align(shrt_pdbn + " and chain " + chain, refpdb)
            rms_super = rms_data_super[0]
            rms_align = rms_data_align[0]
            print("@@@@@ RMS super, align", rms_data_super[0], rms_data_align[0])
            print(shrt_pdbn[:4])
            # With exceptions to SUPER giving larger RMS than ALI but being better
            if rms_align > rms_super or shrt_pdbn[:4] == "6N51":
                rms_data_super = pymol.cmd.super(
                    shrt_pdbn + "////CA", refpdb + "////CA"
                )
            rms_a.append(rms_align)
            rms_s.append(rms_super)
            rms_pdb.append(shrt_pdbn)
            pymol.cmd.do("cd " + "../REC_MOL_WAT")
            pymol.cmd.do("save " + pdbname + ", " + shrt_pdbn)
        except:
            print("Error: " + shrt_pdbn)
            pymol.cmd.do("cd " + "../REC_MOL_WAT")
        # end loop
        l_error_wat.append(shrt_pdbn)
        pymol.cmd.do("delete " + shrt_pdbn)
        pymol.cmd.do("cd ..")
    print(d_waters)
    print("        --- FINISHED --- directories REC_MOL_WAT, REC_WAT, REC  ---")

    # write waters to db
    all_info_rec = pd.read_csv("E:\Work\HomolWat\HomolWat_db.csv", sep=";")
    l_waters = []
    for index, row in all_info_rec.iterrows(): 
        pdb_n = row["PDB accesion"] 
        l_waters.append(d_waters[pdb_n.upper()])
    all_info_rec["Waters"] = l_waters
    all_info_rec.to_csv("E:\Work\HomolWat\HomolWat_db.csv", index=False)

    # write rmsds
    file_rms = open("rmsd.txt", "w")
    rms_sort = list(zip(rms_pdb, rms_s, rms_a))
    rms_sort.sort()
    # rms_sort.reverse()
    print("\n", rms_sort, "\n")
    for rms in rms_sort:
        out = "%s %6.3f %6.3f\n" % (rms[0], rms[1], rms[2])
        file_rms.write(out)
    file_rms.close()
    return l_error_wat

In [ ]:
print(os.getcwd())


In [8]:
import time
# Actual code starts

"""Download and process PDBs associated to "lmcdb pdb"
It will only process new PDBs.
If you want to regenerate old PDBs remove them from the directory FULL.
problem

# For new proteins, check (in the directory FULL) which are the chains that map one receptor.
# Ex: receptor + peptide, receptor + nanobody ...
# In case there is more than "receptor" , edit "pdb_mappings.txt", remove the new PDBs and rerun this
:param machine: where to run the sctip (uab or arnau)
"""
global host, PDB_PATH, pdb_pepts
global pdb_list, d_chains, d_uniprot, d_expmet

#DEFINE PATHS
PDB_PATH = "/Work/HomolWat"
print(os.getcwd())
if not os.path.exists("/Work/HomolWat/PDB"):
    os.mkdir("/Work/HomolWat/PDB")
os.chdir(f"{PDB_PATH}/PDB")

if not os.path.exists("CHAIN"):
    for dir in dirs:
        try:
            os.stat(dir)
        except FileNotFoundError:
            os.mkdir(dir)

pdb_list, d_chains, d_uniprot, d_expmet, d_segments = get_pdb_chains()

# for testing
# pdb_list = ['1GZM', '2RH1', '4DKL']
# d_chains = {'1GZM': ['A', 'B'], '2RH1': ['A'], '4DKL': ['A'],}
# d_uniprot = {'1GZM': 'OPSD', '2RH1': 'ADRB2','4DKL': 'OPRM',}

#pdb_pepts = a_pdb2ball.read_pdb_mapping()
#new_pdbs = prepare_FULL(pdb_list)  # returns the list of only "new" pdb
new_pdbs = os.listdir("/Work/HomolWat/PDB/FULL")
time.sleep(1)
pymol.finish_launching()  # maybe ['pymol', '-q'])
#prepare_CHAIN(new_pdbs, d_chains, d_uniprot)  # CHANGE new_pdbs, pdb_list
l_error_wat = prepare_REC(new_pdbs, d_segments)  # CHANGE new_pdbs, pdb_list

pymol.cmd.quit()
print(l_error_wat)

Ara falten les seq��ncies en format Fasta des d'Uniprot. 

In [8]:
import pandas as pd
import re
import requests
import json
all_info_rec = pd.read_csv("E:\Work\HomolWat\HomolWat_db.csv", sep=",")
columns_recp = ["PDB accesion", "Resolution", "Method", "Id", "Uniprot accesion", "Uniprot entry", "Pfam accesion", "Chain", "Segments Pdb", "Classification", "Family", "Waters", "Fasta"]
l_fasta = []
l_uni = all_info_rec["Uniprot accesion"].to_list()
for uni in  l_uni:
    url_uni = f'https://www.uniprot.org/uniprot/{uni}.fasta'
    response = requests.get(url_uni)
    data_uni = response.text
    l_fas = data_uni.split("\n")
    fas = "".join(l_fas[1:])
    l_fasta.append(fas)
all_info_rec["Fasta"] = l_fasta
all_info_rec = all_info_rec[columns_recp]
all_info_rec.to_csv("E:\Work\HomolWat\HomolWat_db.csv", index=False)